In [2]:
from cdparser import Classifier, Features, LabeledEntry, Utils
import json
import ast

In [3]:
## Create a classifier object and load some labeled data from a CSV
classifier = Classifier.Classifier()
classifier.load_training("data/nyc-city-directories/directory-specific.csv")

In [4]:
## Optionally, load validation dataset
# classifier.load_validation("/full/path/to/validation/nypl-labeled-validate.csv")

## Train your classifier (with default settings)
classifier.train()

In [5]:
#read cleaned data
with open(r"MN_1850-51_cleaned.txt","r",encoding = "utf-8") as f:
    lines = f.readlines()

In [6]:
#remove newlines
lines = [x.strip() for x in lines] 

In [7]:
entry_dict = {}
for i in range(len(lines)):
## Create an entry object from string
    entry = LabeledEntry.LabeledEntry(lines[i])

    ## Pass the entry to the classifier
    classifier.label(entry)

    ## Export the labeled entry as JSON
    entry_dict[str(i+1)] = json.dumps(entry.categories)
# with open('CRF_output.txt', 'a') as fout:
#         json.dump(entry_dict,fout)

In [8]:
cities = ['Hoboken', 'N. J.', 'Spain', 'B’klyn', 'Mt. Vernon', 'Jersey City', 'Brooklyn']
def has_city(address, cities = cities): #returns city if present
    for city in cities:
        if city in address:
              return city
    return ''

In [44]:
full_entry =[]
for i in range(len(lines)):
    full_entry.append(str(i+1)+ ' '+lines[i])
full_entry

['1 Ackley Addison, clerk, 47 Broadway, h. 39 Pearl',
 '2 Adams William, weigher, 20 Washington & 13 West, h. 170 Clinton, Brooklyn',
 '3 Aitkin Thomas, stonecutter, 601 Washington',
 '4 Albro Benjamin sen. 328 Bowery',
 '5 Allen E. W. publisher, 42 Orange',
 '6 Anderson Frederick R. com. mer. 13 Front, h. 77 Barrow',
 '7 Anderson John R. seaman, 586 E, 4th',
 '8 Anthes Peter, barbex 34 Av. C',
 '9 Anthes & Eckardt, barbers, 34 Av. C',
 '10 Appell John, tailor, 40 Av. C',
 '11 Arcularius Jacob S. h. 175 Canal',
 '12 Arenfred George, 137 Norfolk',
 '13 Armstrong George, shoemaker, 53 Av. D',
 '14 Armstrong John D. coachmaker, 5 Ludlow',
 '15 Arnold Henry, drygoods, 1 Av. D, h. r. 1 Av. D',
 '16 Ash William A. saloon, 413 B.way, h. Morrisania',
 '17 Auchincloss John T. com. mer. 192 Fulton, h. 7 Murray',
 '18 Auchincloss & Croxon, com. mers. 192 Fulton',
 '19 Bagley William, clerk, bank, 50 Wall, h. 91 Henry, Brooklyn',
 '20 BAIN ALEXANDER, telegraph 181 inventor,',
 '21 Baldwin Ezekiel,

In [99]:
n = len(entry_dict)
final_output = {}
# generate fields
for i in range(1,n+1):
    cd = {}
    cd['Full Name'] = ''
    cd['First Name'] = ''
    cd['Last Name'] = ''
    cd['Middle Name'] = ''
    cd['Widow-flag'] = 'False'
    cd['Widow Of'] = ''
    cd['Title'] = ''
    cd['Occupation'] = ''
    cd['Full Address'] = ''
    cd['H Status-flag'] = ''
    cd['H Address'] = ''
    cd['H House Number'] = ''
    cd['H Street Name'] = ''
    cd['H City'] = ''
    cd['Address 2'] = ''
    cd['Address 2 House Number'] = ''
    cd['Address 2 Street Name'] = ''
    cd['Address 2 City'] = ''
#     full_entry_ls = full_entry[i-1].split()
#     cd['Index'] = full_entry_ls[0]
#     cd['Full Entry'] = ' '.join(full_entry_ls[1:])

    record = entry_dict[str(i)]
    # name
    try:
        fullname = ast.literal_eval(entry_dict[str(i)])['subjects'][0]
        fullname_list = fullname.split()
        cd['Full Name'] = fullname
        if(len(fullname_list) == 2):
            cd['Last Name'] = fullname_list[0]
            cd['First Name'] = fullname_list[1]
        elif(len(fullname_list) == 3):
            cd['Last Name'] = fullname_list[0]
            cd['First Name'] = fullname_list[1]
            cd['Middle Name'] = fullname_list[2]
#         elif(fullname_list[2] == '.' or fullname_list[2] == ','):
#             cd['Last Name'] = fullname_list[0]
#             cd['First Name'] = fullname_list[1]
#             cd['Middle Name'] = fullname_list[3]
            
        
        
        occupation_to_check_for_widows =  ast.literal_eval(entry_dict[str(i)])['occupations'][0]

    #widow flag
        if('wid' in fullname or 'wid' in occupation_to_check_for_widows):
                cd['Widow-flag'] = 'True'
    except:
        pass
  
  #occupation
    try:
        occupation_list =  ast.literal_eval(entry_dict[str(i)])['occupations']
        if(len(occupation_list) == 1):
            cd['Occupation'] = occupation_list[0]
        else:
            cd['Occupation'] = occupation_list[1]
    except:
        pass
  
#   #address
    try:
        len(ast.literal_eval(entry_dict['3'])['locations'])
        address = record['Address']
        cd['Full Address'] = ', '.join(address)
        if(len(address) == 1): #this means it is house address
            cd['H Address'] = address[0]
            address_1 = address[0].split()
            if ((address_1[0] == 'h') or (address_1[0] == 'h.')): #in h
                cd['H Status-flag'] = 'True'
                start = 1
            else:
                cd['H Status-flag'] = 'Assume'
                start = 0 #index if string where address starts
      
            house_num = False #finds starting of house nmber
            if(address_1[start].isdigit()):
                house_num = True
            elif(address_1[start+1].isdigit()):
                house_num = True
                start += 1

            if(house_num):
                cd['H House Number'] = address_1[start]
        #city
                city = has_city(cities, address_1)
                if(city):
                    cd['H City'] = city
                    cd['H Street Name'] = ' '.join(address_1[(start+1):]).replace(city,'').strip() #everythign except the city 
                else:
                    cd['H Street Name'] = ' '.join(address_1[(start+1):])
            else: #everything goes into city
                cd['H City'] = ' '.join(address_1[start:])

        else: #len of address >1, then 2nd add is house. 1st is other address.
            add = address[0]
            add_ls = add.split()
            cd['Address 2'] = add
            start = 0
            house_num = False
            if(add_ls[start].isdigit()):
                house_num = True
            elif(add_ls[start+1].isdigit()):
                house_num = True
                start += 1
            if(house_num):
                cd['Address 2 House Number'] = add_ls[start]
            #city
                city = has_city(add)
                if(city):
                    cd['Address 2 City'] = city
                    cd['Address 2 Street Name'] = ' '.join(add_ls[(start+1):]).replace(city, '').strip()
                else:
                    cd['Address 2 Street Name'] = ' '.join(add_ls[(start+1):])
            else:
                cd['Address 2 City'] = ' '.join(add_ls[start:])
      
            add_1 = address[1]
            add_ls = add_1.split()
            if ((add_ls[0] == 'h') or (add_ls[0] == 'h.')):
                cd['H Address'] = add_1
                cd['H Status-flag'] = 'True'
                start = 1
                house_num = False
                if(add_ls[start].isdigit()):
                    house_num = True
                elif(add_ls[start+1].isdigit()):
                    house_num = True
                    start += 1
                if(house_num):
                    cd['H House Number'] = add_ls[start]
              #city
                    city = has_city(add_1)
                    if(city):
                        cd['H City'] = city
                        cd['H Street Name'] = ' '.join(add_ls[(start+1):]).replace(city, '').strip()
                    else:
                        cd['H Street Name'] = ' '.join(add_ls[(start+1):])
                else:
                    cd['H City'] = ' '.join(add_ls[start:])

    except:
        pass

    final_output[str(i)] = cd

In [100]:
final_output

{'1': {'Full Name': 'Ackley Addison',
  'First Name': 'Addison',
  'Last Name': 'Ackley',
  'Middle Name': '',
  'Widow-flag': 'False',
  'Widow Of': '',
  'Title': '',
  'Occupation': 'clerk',
  'Full Address': '',
  'H Status-flag': '',
  'H Address': '',
  'H House Number': '',
  'H Street Name': '',
  'H City': '',
  'Address 2': '',
  'Address 2 House Number': '',
  'Address 2 Street Name': '',
  'Address 2 City': ''},
 '2': {'Full Name': 'Adams William',
  'First Name': 'William',
  'Last Name': 'Adams',
  'Middle Name': '',
  'Widow-flag': 'False',
  'Widow Of': '',
  'Title': '',
  'Occupation': 'weigher',
  'Full Address': '',
  'H Status-flag': '',
  'H Address': '',
  'H House Number': '',
  'H Street Name': '',
  'H City': '',
  'Address 2': '',
  'Address 2 House Number': '',
  'Address 2 Street Name': '',
  'Address 2 City': ''},
 '3': {'Full Name': 'Aitkin Thomas',
  'First Name': 'Thomas',
  'Last Name': 'Aitkin',
  'Middle Name': '',
  'Widow-flag': 'False',
  'Widow O

In [89]:
#checking widows 
text_file = open("widows_utf8.txt", "r", encoding = "utf8")
prev_widows = text_file.read()


In [69]:
new_widows = []
for i in range(1,len(final_output)):
    if(final_output[str(i)]['Widow-flag']):
        new_widows.append(final_output[str(i)]['Full Name'])
new_widows


['Ackley Addison',
 'Adams William',
 'Aitkin Thomas',
 'Albro Benjamin',
 'Allen E . W',
 'Anderson Frederick R',
 'Anderson John R',
 'Anthes Peter',
 'Anthes & Eckardt',
 'Appell John',
 'Arcularius Jacob S',
 'Arenfred George',
 'Armstrong George',
 'Armstrong John D',
 'Arnold Henry',
 'Ash William A',
 'Auchincloss John T',
 'Auchincloss & Croxon',
 'Bagley William',
 'BAIN ALEXANDER',
 'Baldwin Ezekiel',
 'Ball Isaac',
 'Bannon Michael',
 'Barmore Christopher',
 'Barratt Thomas',
 'Barrowclough James , E',
 'Barry James',
 'Barry John',
 'Bates John C',
 'Beach John T',
 'Bear Isaac',
 'Beard William',
 'Becom John J',
 'Beebe Joseph D',
 'Beebe & Co',
 'Beers William P',
 'Bender George W',
 'Benneit Cornelius',
 'Bennett George',
 'Bennett William',
 'Bernhard Gustavus',
 'Bernz John',
 'Berrian Burr',
 'Bird George L',
 'Bill Hannah',
 'Birmingham David',
 'Boole Thomas B',
 'Borman Eckhard',
 'Bott John',
 'Botz Louis',
 'Bouton Gilbert',
 'Bowen John',
 'Boyd John jr',
 'Bo

In [ ]:
new_found_widows = []
for widow in new_widows:
    if widow not in :
    widows.append(final_output[str(i)]['Full Name'])
widows


In [37]:
#names that are not 2 or three words long
n = len(entry_dict)
count = 0
namelengths =[]
for i in range(1,n+1):
    try:
        name = ast.literal_eval(entry_dict[str(i)])['subjects'][0]
        name = name.split()
        if(len(name) != 2 and len(name) != 3):
            print(name, len(name))
            count += 1
            namelengths.append(len(name))
    except:
        pass
print(count)
print(namelengths)

['Allen', 'E', '.', 'W'] 4
['Barrowclough', 'James', ',', 'E'] 4
['Amerman', 'Geo', '.', 'V'] 4
['Amerman', 'O', '.', 'V', '.', '(Rev'] 6
['Amerman', 'Wm', '.', 'C'] 4
['Amswick', 'L', '.', 'E'] 4
['Anderson', 'Chas', '.', 'F'] 4
['Anderson', 'Eliza', 'J', '.', '(col’d)', 'wid', '.', 'Thos'] 8
['Anderson', 'Jas', '.', 'J'] 4
['Anderson', 'James', 'W', '.', 'Broadway'] 5
['Anderson', 'John', '&', 'Co'] 4
['Anderson', 'John', 'H', '.', '&', 'Co'] 6
['Anderson', 'Robert', 'S', '.', 'N'] 5
['Anderson', 'Wm', '.', 'Theodore'] 4
['Andersén', 'C', '.', 'V', '.', '&', 'J'] 7
['Anderson', ',', 'Ritter', '&', 'Co'] 5
['ANDERSON', ',', 'STARR', '&', 'CO', '.'] 6
['Anderton', 'Ralph', '.', 'L'] 4
['Andrews', 'F', '.', 'A'] 4
['Andrews', 'Loring', '&', 'Co'] 4
['Andrews', 'Robt', '.', 'J'] 4
['Andrews', '§', '.', 'B'] 4
['Andrews', 'Thomas', 'M', '.', '15', 'Cortlandt'] 6
['Andrews', 'Wm', '.', 'D'] 4
['Andrews', 'Wm', '.', 'M'] 4
['Andrews', 'B', '.', '&', 'Co'] 5
['ANGEVINE', 'DANIEL', '&', 'LEVI

In [57]:
'Wid' in ast.literal_eval(entry_dict['103'])['occupations'][0]

False

In [107]:
entry_dict

{'1': '{"subjects": ["Ackley Addison"], "occupations": ["clerk"], "locations": [{"value": "47 Broadway"}, {"value": "39 Pearl", "labels": ["h"]}]}',
 '2': '{"subjects": ["Adams William"], "occupations": ["weigher"], "locations": [{"value": "20 Washington & 13 West"}, {"value": "170 Clinton , Brooklyn", "labels": ["h"]}]}',
 '3': '{"subjects": ["Aitkin Thomas"], "occupations": ["stonecutter"], "locations": [{"value": "601 Washington"}]}',
 '4': '{"subjects": ["Albro Benjamin"], "occupations": ["sen"], "locations": [{"value": "328 Bowery"}]}',
 '5': '{"subjects": ["Allen E . W"], "occupations": ["publisher"], "locations": [{"value": "42 Orange"}]}',
 '6': '{"subjects": ["Anderson Frederick R"], "occupations": ["com", "mer"], "locations": [{"value": "13 Front"}, {"value": "77 Barrow", "labels": ["h"]}]}',
 '7': '{"subjects": ["Anderson John R"], "occupations": ["seaman"], "locations": [{"value": "586 E , 4th"}]}',
 '8': '{"subjects": ["Anthes Peter"], "occupations": [], "locations": [{"va